In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Attention, Flatten


In [18]:
# Set random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

In [19]:
# Load data
file_path = '/home/disi/ff/Datasets/Dataset_3/Occupancy_Estimation.csv'
data = pd.read_csv(file_path)


In [44]:
# Define input features and label
input_columns = ['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light', 'S2_Light', 'S3_Light', 'S4_Light',
                 'S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR']
label_column = 'Room_Occupancy_Count'

In [45]:
# Data preprocessing
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data[input_columns])
label = data[label_column].values

In [46]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(scaled, label, test_size=0.3, random_state=42, stratify=label)



In [47]:
# Reshape X_train for the model
X_train = X_train.reshape((X_train.shape[0], 1, len(input_columns)))

In [48]:
# Model configuration
EPOCHS = 50
BATCH = 32
input_shape = (X_train.shape[1], len(input_columns))

In [55]:
# Build the model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape, padding="same"))
model.add(Flatten())
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=input_shape, padding="same"))
model.add(Flatten())
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=input_shape, padding="same"))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
print(model.summary())

ValueError: Input 0 of layer "conv1d_12" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 32)

In [51]:
# Save model path
directory = '/home/disi/ff/Datasets/Dataset_3/'
main_yontem_name = 'CNNdense_'
yontem_name = main_yontem_name
model_path = directory + yontem_name + '.h5'

In [52]:
# Model training
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH, validation_split=0.2, verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=50, mode='min'),
                               keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)])

Epoch 1/50
178/178 [==============================] - 1s 2ms/step - loss: -0.8390 - accuracy: 0.7255 - val_loss: -4.5001 - val_accuracy: 0.7609
Epoch 2/50
122/178 [===================>..........] - ETA: 0s - loss: -14.9925 - accuracy: 0.7049

/home/disi/anaconda3/envs/ff/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


178/178 [==============================] - 0s 2ms/step - loss: -21.5752 - accuracy: 0.6985 - val_loss: -42.4190 - val_accuracy: 0.7306
Epoch 3/50
178/178 [==============================] - 0s 2ms/step - loss: -95.9742 - accuracy: 0.6849 - val_loss: -146.5961 - val_accuracy: 0.7087
Epoch 4/50
178/178 [==============================] - 0s 2ms/step - loss: -264.7285 - accuracy: 0.6809 - val_loss: -349.9749 - val_accuracy: 0.6982
Epoch 5/50
178/178 [==============================] - 0s 2ms/step - loss: -555.5857 - accuracy: 0.6797 - val_loss: -677.7444 - val_accuracy: 0.6996
Epoch 6/50
178/178 [==============================] - 0s 2ms/step - loss: -992.9498 - accuracy: 0.6789 - val_loss: -1140.6007 - val_accuracy: 0.7193
Epoch 7/50
178/178 [==============================] - 0s 2ms/step - loss: -1588.0519 - accuracy: 0.6846 - val_loss: -1753.3234 - val_accuracy: 0.7080
Epoch 8/50
178/178 [==============================] - 0s 2ms/step - loss: -2365.2141 - accuracy: 0.6804 - val_loss: -2535.5

In [54]:
model = Sequential()
model.add(LSTM(input_shape = input_shape[1:], units = 32 ,return_sequences=False))  
model.add(Dense(32, activation = 'relu'))    
model.add(Dense(units=1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 16)